inserting our libraries that need

In [ ]:
import pymongo
import re
from bs4 import BeautifulSoup
import itertools

then maintain a function for connecting to our database for storing our datas
in this project we use MongoDB database

In [25]:
def connect_database():
    conn = pymongo.MongoClient(
            'localhost',
             27017
    )
    db = conn['houses']
    collection = db['berlin']
    return collection

then create a function that scraping our datas that saved from web in html files text where we find in #pages file
saving datas from web in html files have been done in pages.ipynb

In [26]:
def web_scrape():    
    final_houses= []
    for i in range(1,42):
        houses=[]
        with open('pages/web%i.html' %(i), 'rb') as f:

            soup= BeautifulSoup(f.read(), 'html.parser')
            houses_info_tag= soup.find_all(
                'div',
                attrs={'data-test-locator': 'ListingCardPropertyInfo'}
            )

            houses_price_tag= soup.find_all(
                'div',
                attrs={'data-test-locator': 'ListingCardInfoPrice' }
            )

            for j in range(len(houses_info_tag)):
                house= []
                house_info= houses_info_tag[j].text
                house_info_up= house_info.split('•')
                house_price= houses_price_tag[j].text
                type_of_home= house_info_up[0].strip()
                size= (re.findall(r'\d+', house_info_up[1]))[0]
                bedrooms= house_info_up[2][0]
                date= house_info_up[3][5:]
                price= re.findall(r'\d+', house_price)
                if len(price)> 1:
                    final_price=str(int((int(price[0])+ int(price[1]))/2))
                else:
                    final_price= price[0]
                house= [size, bedrooms, final_price, date, type_of_home]
                houses.append(house)
        final_houses.extend(houses)
    final_houses.sort()
    final_houses= list(k for k,_ in itertools.groupby(final_houses))
    return final_houses

now create a function for inserting scraped data in our database

In [27]:
def insert_in_mongodb(final_houses, collection):
    for every in final_houses:
        data= {}
        data['size']= every[0]
        data['number of bedrooms']= every[1]
        data['price']= every[2]
        data['date']= every[3]
        data['type of home']= every[4]
        collection.insert_one(data)
    return

now using our functions

In [29]:
insert_in_mongodb(web_scrape(), connect_database())